In [1]:
import numpy as np
import pandas as pd
import requests
import re
import datetime

import locale
locale.setlocale(locale.LC_TIME, "es_ES") #

'es_ES'

In [2]:
fecha = "18-11-2022"
fecha_anterior = "09-10-2022"
columns = ['Subject','Start Date', 'Start Time', 'End Date','End Time','All Day Event', 'Description','Location','Private']
# 05/30/20
# 10:00 AM

# Cargar Datos

In [3]:
guia = pd.read_csv("../Guia-Teatral-Bogota/data/guia_teatral({}).csv".format(fecha))

In [4]:
guia.sort_values(by='fecha')

,tipo,titulo,fecha,url,duracion,preciomin,preciomax,lugar,desc
218,Improvisación ​,Volver,2022-10-27 20:00,https://www.lamalditavanidadteatro.com/volver,1.00,38000.0,NaN,La Maldita Vanidad,El escenario nos reúne con la intención de VO...
219,Improvisación ​,Volver,2022-10-28 20:00,https://www.lamalditavanidadteatro.com/volver,1.00,38000.0,NaN,La Maldita Vanidad,El escenario nos reúne con la intención de VO...
220,Improvisación ​,Volver,2022-10-29 20:00,https://www.lamalditavanidadteatro.com/volver,1.00,38000.0,NaN,La Maldita Vanidad,El escenario nos reúne con la intención de VO...
230,Improvisación ​,Volver,2022-10-30 18:30,https://www.lamalditavanidadteatro.com/volver,1.00,38000.0,NaN,La Maldita Vanidad,El escenario nos reúne con la intención de VO...
221,Improvisación ​,Volver,2022-11-03 20:00,https://www.lamalditavanidadteatro.com/volver,1.00,38000.0,NaN,La Maldita Vanidad,El escenario nos reúne con la intención de VO...
...,...,...,...,...,...,...,...,...,...
168,Teatro,XXXV Festival FAICP,2023-12-16 20:00,https://www.teatromayor.org/es/evento/teatro/x...,1.25,NaN,NaN,Teatro Estudio,En 2023 el Festival Artístico Internacional In...
169,Música,Orquesta Filarmónica de Bogotá – Director asis...,2023-12-17 11:00,https://www.teatromayor.org/es/evento/musica/o...,1.25,80000.0,80000.0,Teatro Mayor Julio Mario Santo Domingo,Como ya es tradición en el Teatro Mayor Julio ...
170,Música,Orquesta Filarmónica de Bogotá – Director asis...,2023-12-18 20:00,https://www.teatromayor.org/es/evento/musica/o...,1.25,80000.0,80000.0,Teatro Mayor Julio Mario Santo Domingo,Como ya es tradición en el Teatro Mayor Julio ...
171,Música,Orquesta Filarmónica de Bogotá – Director asis...,2023-12-19 20:00,https://www.teatromayor.org/es/evento/musica/o...,1.25,80000.0,80000.0,Teatro Mayor Julio Mario Santo Domingo,Como ya es tradición en el Teatro Mayor Julio ...


# Limpieza

### Tipo

In [5]:
guia.tipo = guia.tipo.str.replace(u'\u200b','').str.strip()

Categorías:

- Teatro
- Danza
- Teatro Musical/Multidisciplinar
- Ópera/Música
- Improvisación

In [8]:
guia.tipo.unique()

array(['Música', 'Danza', 'Teatro Musical', 'Teatro', 'Multidisciplinar',
       nan, 'Improvisación'], dtype=object)

In [7]:
mapa_tipos = {"Comedia":"Teatro","Stand Up Comedy":"Teatro","Drama":"Teatro","Autoficción / Teatro documental":"Teatro",\
    "Todos Los Eventos":"Música","Conciertos":"Música","Alquileres":"Música", "Festivales":"Música",\
        "Ballet":"Danza",\
            "Familia":"Teatro Musical","Circo":"Teatro Musical"}

guia.tipo = guia.tipo.apply(lambda t: mapa_tipos[t] if t in mapa_tipos else t)

In [11]:
guia.loc[guia.tipo.isna(),'tipo'] = guia[guia.tipo.isna()]['titulo'].str.lower().apply(lambda t: "Música" if \
    ("concierto" in t or "sinfonía" in t or "gira" in t or "puerto candelaria" in t or "música" in t or "ópera" in t or "orquesta" in t) else \
        ("Danza" if ("ballet" in t or "romeo y julieta" in t or "danza" in t or "milonga" in t) else \
            ("Teatro Musical" if ("familia" in t or "camilo" in t or "guadalupe años" in t or "milonga" in t or "plancha" in t or "cueva del olvido" in t or "rosa elvira" in t) else "Teatro")))

### Duración

In [15]:
guia.loc[guia.url.str.contains('pastelazo'),'titulo'] = 'Pastelazo a La Monalisa'

In [16]:
guia.loc[guia.duracion.isna(),'duracion'] = guia['duracion'].mean()
guia["duracion"] = guia["duracion"].apply(lambda d: round(d,1))

In [17]:
guia = guia.dropna(subset='fecha')
guia = guia.reset_index(drop=True)
guia.sort_values(by='fecha')

,tipo,titulo,fecha,url,duracion,preciomin,preciomax,lugar,desc
218,Improvisación,Volver,2022-10-27 20:00,https://www.lamalditavanidadteatro.com/volver,1.0,38000.0,NaN,La Maldita Vanidad,El escenario nos reúne con la intención de VO...
219,Improvisación,Volver,2022-10-28 20:00,https://www.lamalditavanidadteatro.com/volver,1.0,38000.0,NaN,La Maldita Vanidad,El escenario nos reúne con la intención de VO...
220,Improvisación,Volver,2022-10-29 20:00,https://www.lamalditavanidadteatro.com/volver,1.0,38000.0,NaN,La Maldita Vanidad,El escenario nos reúne con la intención de VO...
230,Improvisación,Volver,2022-10-30 18:30,https://www.lamalditavanidadteatro.com/volver,1.0,38000.0,NaN,La Maldita Vanidad,El escenario nos reúne con la intención de VO...
221,Improvisación,Volver,2022-11-03 20:00,https://www.lamalditavanidadteatro.com/volver,1.0,38000.0,NaN,La Maldita Vanidad,El escenario nos reúne con la intención de VO...
...,...,...,...,...,...,...,...,...,...
168,Teatro,XXXV Festival FAICP,2023-12-16 20:00,https://www.teatromayor.org/es/evento/teatro/x...,1.2,NaN,NaN,Teatro Estudio,En 2023 el Festival Artístico Internacional In...
169,Música,Orquesta Filarmónica de Bogotá – Director asis...,2023-12-17 11:00,https://www.teatromayor.org/es/evento/musica/o...,1.2,80000.0,80000.0,Teatro Mayor Julio Mario Santo Domingo,Como ya es tradición en el Teatro Mayor Julio ...
170,Música,Orquesta Filarmónica de Bogotá – Director asis...,2023-12-18 20:00,https://www.teatromayor.org/es/evento/musica/o...,1.2,80000.0,80000.0,Teatro Mayor Julio Mario Santo Domingo,Como ya es tradición en el Teatro Mayor Julio ...
171,Música,Orquesta Filarmónica de Bogotá – Director asis...,2023-12-19 20:00,https://www.teatromayor.org/es/evento/musica/o...,1.2,80000.0,80000.0,Teatro Mayor Julio Mario Santo Domingo,Como ya es tradición en el Teatro Mayor Julio ...


## Lugar

In [20]:
guia.loc[guia.lugar.str.lower().str.contains('julio mario santo domingo'),'lugar'] = "Teatro Mayor Julio Mario Santo Domingo"
guia.loc[guia.lugar.str.lower().str.contains('teatro estudio'),'lugar'] = "Teatro Estudio Julio Mario Santo Domingo"
guia.loc[guia.lugar.str.lower().str.contains('gaitán'),'lugar'] = "Teatro Jorge Eliecer Gaitán"
guia.loc[guia.lugar.str.lower().str.contains('ensueño'),'lugar'] = "Teatro El Ensueño"
guia.loc[guia.lugar.str.lower().str.contains('petra'),'lugar'] = "Teatro Petra"
guia.loc[guia.lugar.str.lower().str.contains('castellana'),'lugar'] = "Teatro Nacional La Castellana"
guia.loc[guia.lugar.str.lower().str.contains('villa mayor'),'lugar'] = "Teatro Villa Mayor"
guia.loc[guia.lugar.str.lower().str.contains('libre de chapinero'),'lugar'] = "Teatro Libre Chapinero"
guia.loc[guia.lugar.str.lower().str.contains('cinemateca'),'lugar'] = "Cinemateca de Bogotá"
guia.loc[guia.lugar.str.lower().str.contains('casa del teatro nacional'),'lugar'] = "La Casa del Teatro Nacional"


In [21]:
guia["lugar"] = guia["lugar"].apply(lambda x: x.split('-')[0].strip().split('Calle')[0].strip().split('Carrera')[0].strip().split('Cra')[0].strip().split('calle')[0].strip().split('Cl')[0].strip())

# Exportar

In [139]:
export = guia.copy()
export["Subject"] = export["titulo"]
export["fecha"] = export["fecha"].apply(lambda f: f.replace(":00:00",":00").replace(":30:00",":30"))

In [188]:
export['Start Date'] = export["fecha"].apply(lambda z: datetime.datetime.strptime(z,'%Y-%m-%d %H:%M').strftime('%m/%d/%Y'))
export['Start Time'] = export["fecha"].apply(lambda z: datetime.datetime.strptime(z,'%Y-%m-%d %H:%M').strftime('%I:%M %p'))
export['End Date'] = export["fecha"].apply(lambda z: datetime.datetime.strptime(z,'%Y-%m-%d %H:%M').strftime('%m/%d/%Y'))
export['End Time'] = export.apply(lambda f: (datetime.datetime.strptime(f["Start Time"],'%I:%M %p')+datetime.timedelta(hours=f["duracion"])).strftime('%I:%M %p'), axis=1)

In [195]:
export['Location'] = export['lugar']
export['Description'] = export['tipo'] +'\n'+export['preciomin'].apply(lambda p: "{}k".format(int(p/1000)) if p>0 else "0")+'\n'+export['desc'].apply(lambda d: str(d).replace('\n',''))+'\n'+export['url']

In [196]:
columns

['Subject',
 'Start Date',
 'Start Time',
 'End Date',
 'End Time',
 'All Day Event',
 'Description',
 'Location',
 'Private']

In [197]:
export['All Day Event'] = False
export['Private'] = False

In [198]:
export.tipo.unique()

array(['Música', 'Teatro', 'Danza', 'Teatro Musical', 'Improvisación',
       'Festival'], dtype=object)

In [202]:
export[export.tipo=="Festival"][columns].to_csv("./export/festival{}.csv".format(fecha),index=False)